# AUDIO WRANGLING 🔊

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pydub.utils import make_chunks
from pydub import AudioSegment
from scipy.fftpack import fft
from pydub.utils import make_chunks
import numpy as np
import pandas as pd
import re
import pickle

----

In [2]:
path = "../input"
for familia in os.listdir(path):
    familia = os.path.join(path, familia)
    print(familia)

../input/.DS_Store
../input/copy.mp3
../input/recording_copy.mp3
../input/Bufonidae
../input/Arthroleptidae
../input/recording.mp3
../input/Hyperollidae
../input/Leptodactylidae
../input/Dendrobatidae


In [3]:
def audioSegmentData(direc, chunk_length_ms):
    fam = []
    path = f"../input/{direc}"
    for audio in os.listdir(path):
        if audio.endswith('.mp3'):
            croak = AudioSegment.from_mp3(path+"/"+audio)
            croak = croak.set_frame_rate(16000)
            
            croak_chunks = make_chunks(croak, chunk_length_ms)
            fam.append(croak_chunks)
    return fam

---

## Segmenting Recordings

In [4]:
buf = audioSegmentData("Bufonidae", 5000)
buf = [x for e in buf for x in e]
len(buf)

444

In [5]:
den = audioSegmentData("Dendrobatidae", 5000)
den = [x for e in den for x in e]

In [6]:
hyp = audioSegmentData("Hyperollidae", 5000)
hyp = [x for e in hyp for x in e]

In [7]:
lep = audioSegmentData("Leptodactylidae", 5000)
lep = [x for e in lep for x in e]

-------

------

## Extracting Fourier Features

In [8]:
def audioFeaturesFourier(aud):
    arr = aud.get_array_of_samples()[:5120]
    abs_four = np.abs(fft(arr))
    return abs_four

In [9]:
bufFF = [audioFeaturesFourier(chunk) for chunk in buf]
len(bufFF)

444

In [10]:
denFF = [audioFeaturesFourier(chunk) for chunk in den]
len(denFF)

349

In [11]:
hypFF = [audioFeaturesFourier(chunk) for chunk in hyp]
len(hypFF)

275

In [12]:
lepFF = [audioFeaturesFourier(chunk) for chunk in lep]
len(lepFF)

380

### All vectors must have the same length, the shortest of them all is 5120.

In [13]:
x = [len(e) for e in denFF]
print(min(x))
print(max(x))

5120
5120


-----

----

## Calculating the Average Value of each MFCC Coefficient

### Mel-Frequency Cepstral Coefficients

In [14]:
from librosa.feature import mfcc

In [21]:
def mfccCoefficients(arr):
    mfcC = mfcc(y=np.array(arr).astype('float64'),sr=len(sample), n_mfcc=256)
    return np.mean(mfccC, axis=1)

In [ ]:
bufccC = mfccCoefficients(buf)
bufccC

In [ ]:
denccC = mfccCoefficients(den)
denccC

In [ ]:
hypccC = mfccCoefficients(hyp)
hypccC

In [ ]:
lepccC = mfccCoefficients(lep)
lepccC

---

## Visualizing the MFCC Series

In [ ]:
with plt.xkcd():
    f, ax = plt.subplots(1,4, figsize = (15,4))
    librosa.display.specshow(mfccs, x_axis='time')
    plt.colorbar()
    plt.title('MFCC')
    plt.tight_layout()

In [ ]:
with plt.xkcd():
    f, ax = plt.subplots(1,4, figsize = (15,4))
    librosa.display.specshow(mfccsB, ax = ax[0])
    librosa.display.specshow(mfccsD, ax = ax[1])
    librosa.display.specshow(mfccsH, ax = ax[2])
    librosa.display.specshow(mfccsL, ax = ax[3])
    
    sns.countplot(train_dum['Tag_2'], ax = ax[1])

---

---

## Creating *Familia* Dataframes

In [14]:
buf_or = {"F.Transform": [a for a in bufFF], "Tag": 1}
den_or = {"F.Transform": [a for a in denFF], "Tag": 2}
hyp_or = {"F.Transform": [a for a in hypFF], "Tag": 3}
lep_or = {"F.Transform": [a for a in lepFF], "Tag": 4}

In [16]:
w = pd.DataFrame(buf_or)
x = pd.DataFrame(den_or)
h = pd.DataFrame(hyp_or)
z = pd.DataFrame(lep_or)

totus = pd.concat([w, x, h, z]).reset_index(drop=True)
totus

,F.Transform,Tag
0,"[214.0, 1468.2663139035296, 1354.9901292253478...",1
1,"[2884.0, 28338.15576414532, 62698.33809773314,...",1
2,"[231582.0, 274082.53149146156, 349585.41389145...",1
3,"[79700.0, 102571.46322235715, 153200.915277075...",1
4,"[63870.0, 52903.555252298946, 141644.395098856...",1
...,...,...
1443,"[583.0, 15302.869133939248, 8141.018024000418,...",4
1444,"[7528.0, 36567.48880158042, 32709.202174082795...",4
1445,"[57975.0, 49892.81312863944, 28033.09150727976...",4
1446,"[11773.0, 12848.633686627703, 17174.7713207563...",4


In [22]:
# w.to_pickle('../output/fam_df_buf.csv')

In [23]:
# x.to_pickle('../output/fam_df_den.csv')

In [17]:
# h.to_pickle('../output/fam_df_hyp.csv')

In [25]:
# z.to_pickle('../output/fam_df_lep.csv')

In [ ]:
# totus.to_pickle('../output/fam_df.csv')

## Classifying with get_dummies

In [19]:
dum_totus = pd.get_dummies(totus, prefix=['Tag'], columns=['Tag'])
dum_totus

,F. Transform,Tag_1,Tag_2,Tag_3,Tag_4
0,18001.000000,1,0,0,0
1,2077.438012,1,0,0,0
2,4324.447678,1,0,0,0
3,7462.545756,1,0,0,0
4,8746.229687,1,0,0,0
...,...,...,...,...,...
364827,190248.712618,0,0,0,1
364828,118798.087221,0,0,0,1
364829,129512.736967,0,0,0,1
364830,202574.367594,0,0,0,1


----

---

## Exporting the df to csv

In [59]:
# dum_totus.to_csv('../output/fam_dum_df.csv')